In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregconsult').getOrCreate()
data=spark.read.csv('customer_churn.csv',inferSchema=True,header=True)
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [4]:
data.show(3)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 3 rows



In [6]:
data.describe().toPandas()

,summary,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Location,Company,Churn
0,count,900,900,900,900,900,900,900,900,900
1,mean,None,41.81666666666667,10062.82403333334,0.4811111111111111,5.27315555555555,8.587777777777777,None,None,0.16666666666666666
2,stddev,None,6.127560416916251,2408.644531858096,0.4999208935073339,1.274449013194616,1.7648355920350969,None,None,0.3728852122772358
3,min,Aaron King,22.0,100.0,0,1.0,3.0,"00103 Jeffrey Crest Apt. 205 Padillaville, IA ...",Abbott-Thompson,0
4,max,Zachary Walsh,65.0,18026.01,1,9.15,14.0,Unit 9800 Box 2878 DPO AA 75157,"Zuniga, Clark and Shaffer",1


In [8]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [10]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')
output=assembler.transform(data)
final_data=output.select('features','churn')

In [11]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [13]:
from pyspark.ml.classification import LogisticRegression
lr_churn=LogisticRegression(featuresCol='features',labelCol='churn')

In [15]:
model=lr_churn.fit(train_data)
summary=model.summary

In [18]:
summary.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[5.08960749037025...|[0.99387728118255...|       0.0|
|[25.0,9672.03,0.0...|  0.0|[5.27230059510233...|[0.99489441001058...|       0.0|
|[26.0,8787.39,1.0...|  1.0|[0.64235211430291...|[0.65528496554680...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.99585387374680...|[0.99908516709364...|       0.0|
|[28.0,11204.23,0....|  0.0|[1.99811921796504...|[0.88059946645828...|       0.0|
|[28.0,11245.38,0....|  0.0|[4.37503440512508...|[0.98756877153505...|       0.0|
|[29.0,5900.78,1.0...|  0.0|[4.30413088167927...|[0.98666753212289...|       0.0|
|[29.0,9617.59,0.0...|  0.0|[4.94665163553668...|[0.99294298883731...|       0.0|
|[29.0,12711.15,0....|  0.0|[6.07640914907019...|[0.99770885348304...|       0.0|
|[30.0,6744.87,0

In [19]:
#evaluate for test data
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred_label=model.evaluate(test_data)

In [25]:
pred_label.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,1.0,...|    0|[5.86277176207306...|[0.99716471007027...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.42643716230658...|[0.99978104758583...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.62074486150074...|[0.83489782969769...|       0.0|
|[28.0,11128.95,1....|    0|[4.52050844814239...|[0.98923368647820...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.87587780708307...|[0.94664102719765...|       0.0|
|[29.0,9378.24,0.0...|    0|[5.23552636271739...|[0.99470417371257...|       0.0|
|[29.0,10203.18,1....|    0|[4.09863228096457...|[0.98367555237384...|       0.0|
|[29.0,11274.46,1....|    0|[4.78729886365657...|[0.99173395617167...|       0.0|
|[29.0,13240.01,1....|    0|[7.29670713655446...|[0.99932269215826...|       0.0|
|[29.0,13255.05,

In [33]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')
auc = eval.evaluate(pred_label.predictions)
auc

0.7553073861123398

In [34]:
final_lr_model = lr_churn.fit(final_data)

In [38]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [39]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [40]:
test_new_cust=assembler.transform(new_customers)

In [42]:
final_results=model.transform(test_new_cust)

In [44]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

